<a href="https://colab.research.google.com/github/Ash100/Minor/blob/main/half-RNAseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install GEOparse biopython pandas matplotlib seaborn

In [ ]:
import GEOparse

# Load the GEO dataset
gse = GEOparse.get_GEO(geo="GSE266315", destdir="/content/sample_data")
print(gse)

In [ ]:
# Inspect the first GSM sample to check its table structure
first_gsm = list(gsm_list.values())[0]
print(first_gsm.table.head())  # This will show the first few rows of the GSM table

Empty DataFrame
Columns: []
Index: []


In [ ]:
# Check the number of GSM samples loaded
print(f"Number of GSM samples: {len(gsm_list)}")

# Check the first GSM object
first_gsm = list(gsm_list.values())[0]
print(first_gsm)

Number of GSM samples: 21
<SAMPLE: GSM7164231>


In [ ]:
# Explore the structure of the first GSM object
first_gsm = list(gsm_list.values())[0]

# Check available metadata
print(first_gsm.metadata)

# Check if there are any associated supplementary files
print("Supplementary files:", first_gsm.metadata.get("supplementary_file"))

# Check if the expression data is in another attribute
print("Tables available in GSM object:", first_gsm.tables)  # If there's more than one table

In [ ]:
!apt-get install sra-toolkit

In [ ]:
!fastq-dump --split-files SRR24142133

Read 25814203 spots for SRR24142133
Written 25814203 spots for SRR24142133


In [ ]:
# Install HISAT2
!apt-get install -y hisat2

# Install SAMtools (for handling SAM/BAM files)
!apt-get install -y samtools

# Install featureCounts (for counting reads)
!apt-get install -y subread

In [ ]:
!wget ftp://ftp.ensembl.org/pub/release-109/fasta/mus_musculus/dna/Mus_musculus.GRCm39.dna.primary_assembly.fa.gz
!gunzip Mus_musculus.GRCm39.dna.primary_assembly.fa.gz

# Download gene annotation (GTF file) for Mus musculus
!wget ftp://ftp.ensembl.org/pub/release-109/gtf/mus_musculus/Mus_musculus.GRCm39.109.gtf.gz
!gunzip Mus_musculus.GRCm39.109.gtf.gz

In [ ]:
# Build HISAT2 index
!hisat2-build Mus_musculus.GRCm39.dna.primary_assembly.fa mus_musculus_index

In [ ]:
# Example alignment command
!hisat2 -x mus_musculus_index -1 SRR24142133_1.fastq -2 SRR24142133_2.fastq -S output.sam

25814203 reads; of these:
  25814203 (100.00%) were paired; of these:
    3265035 (12.65%) aligned concordantly 0 times
    21435472 (83.04%) aligned concordantly exactly 1 time
    1113696 (4.31%) aligned concordantly >1 times
    ----
    3265035 pairs aligned concordantly 0 times; of these:
      258452 (7.92%) aligned discordantly 1 time
    ----
    3006583 pairs aligned 0 times concordantly or discordantly; of these:
      6013166 mates make up the pairs; of these:
        4044561 (67.26%) aligned 0 times
        1783461 (29.66%) aligned exactly 1 time
        185144 (3.08%) aligned >1 times
92.17% overall alignment rate


In [ ]:
# Convert SAM to BAM
!samtools view -S -b output.sam > output.bam

# Sort BAM file
!samtools sort output.bam -o output_sorted.bam

[bam_sort_core] merging from 21 files and 1 in-memory blocks...


In [ ]:
!featureCounts -p -a Mus_musculus.GRCm39.109.gtf -o counts.txt output_sorted.bam


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.3

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 1 BAM file                                       ||
||                                                                            ||
||                           output_sorted.bam                                ||
||                                                                            ||
||             Output file : counts.txt                          

In [ ]:
import pandas as pd

# Load the counts file
counts = pd.read_csv("counts.txt", sep="\t", comment="#", index_col=0)

# Show the first few rows of the count data
counts.head()

,Chr,Start,End,Strand,Length,output_sorted.bam
Geneid,,,,,,
ENSMUSG00000104478,1,108344807,108347562,+,2756,0
ENSMUSG00000104385,1,6980784,6981446,+,663,0
ENSMUSG00000086053,1;1,75368775;75372851,75369089;75373007,-;-,472,2
ENSMUSG00000101231,1,108540067,108540244,-,178,0
ENSMUSG00000102135,1;1,6986783;6993624,6987219;6993812,+;+,626,52


In [ ]:
# Install R in Colab
!apt-get install -y r-base

# Install required R packages for RNA-seq analysis
!R -e 'install.packages(c("BiocManager"), repos="http://cran.rstudio.com/")'
!R -e 'BiocManager::install("DESeq2")'
!R -e 'install.packages("ggplot2", repos="http://cran.rstudio.com/")'

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R

# Load necessary libraries
library(DESeq2)
library(ggplot2)

# Load the counts data
counts <- read.csv("counts.txt", sep="\t", comment.char="#", row.names=1)

# Load metadata (e.g., experimental conditions)
metadata <- data.frame(
    row.names = colnames(counts),
    condition = c("control", "control", "treated", "treated")  # example conditions
)

print("Data and metadata loaded successfully.")

In [ ]:
%%R
# Check the dimensions of the counts matrix
print(dim(counts))  # Should output (number of genes, number of samples)

# Check if row names (genes) and column names (samples) are correctly set
print(rownames(counts))  # Should output gene names
print(colnames(counts))  # Should output sample names

Streaming output truncated to the last 5000 lines.
[42019] "ENSMUSG00000038732" "ENSMUSG00000084669" "ENSMUSG00000082648"
[42022] "ENSMUSG00000037851" "ENSMUSG00000087963" "ENSMUSG00000077360"
[42025] "ENSMUSG00000049115" "ENSMUSG00000113424" "ENSMUSG00000118398"
[42028] "ENSMUSG00000083882" "ENSMUSG00000113823" "ENSMUSG00000114073"
[42031] "ENSMUSG00000038462" "ENSMUSG00000112995" "ENSMUSG00000086263"
[42034] "ENSMUSG00000069236" "ENSMUSG00000113254" "ENSMUSG00000113530"
[42037] "ENSMUSG00000113333" "ENSMUSG00000113782" "ENSMUSG00000093976"
[42040] "ENSMUSG00000094886" "ENSMUSG00000114175" "ENSMUSG00000090497"
[42043] "ENSMUSG00000113963" "ENSMUSG00000113312" "ENSMUSG00000113888"
[42046] "ENSMUSG00000113007" "ENSMUSG00002076947" "ENSMUSG00000113921"
[42049] "ENSMUSG00000113227" "ENSMUSG00000113437" "ENSMUSG00000113718"
[42052] "ENSMUSG00000113480" "ENSMUSG00000069255" "ENSMUSG00000081905"
[42055] "ENSMUSG00000095300" "ENSMUSG00002076704" "ENSMUSG00000114035"
[42058] "ENSMUSG0000010113

In [ ]:
%%R

# Create DESeq2 dataset
dds <- DESeqDataSetFromMatrix(countData = counts, colData = metadata, design = ~ condition)

# Pre-filter to remove genes with low counts (optional but recommended)
dds <- dds[rowSums(counts(dds)) > 10, ]

# Print the DESeq2 dataset summary
print(dds)

  attempt to set 'rownames' on an object with no dimensions




Error in `rownames<-`(`*tmp*`, value = colnames(countData)) : 
  attempt to set 'rownames' on an object with no dimensions


In [ ]:
# In terminal or using Colab bash commands
!hisat2 -p 4 -x genome_index -U reads.fastq.gz -S output.sam

In [ ]:
!samtools view -bS output.sam > output.bam

In [ ]:
!featureCounts -T 4 -t exon -g gene_id -a annotation.gtf -o counts.txt aligned_reads.bam

In [ ]:
# Install required libraries in R
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("DESeq2")

# Load DESeq2
library(DESeq2)

# Read in count matrix
counts <- read.csv("counts.txt", row.names = 1)

# Create DESeq2 dataset
dds <- DESeqDataSetFromMatrix(countData = counts, colData = colData, design = ~ condition)

# Normalize counts
dds <- DESeq(dds)
normalized_counts <- counts(dds, normalized = TRUE)

In [ ]:
# Perform differential expression analysis
res <- results(dds)

# Summarize results
summary(res)

# Filter significant genes
sig_res <- subset(res, padj < 0.05)

In [ ]:
# Install and load clusterProfiler
BiocManager::install("clusterProfiler")
library(clusterProfiler)

# Perform GO enrichment analysis
ego <- enrichGO(gene = rownames(sig_res), OrgDb = org.Hs.eg.db, keyType = "ENSEMBL", ont = "BP")
head(ego)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns

# PCA on normalized counts
pca = PCA(n_components=2)
pca_result = pca.fit_transform(normalized_counts.T)

# Plot PCA
plt.figure(figsize=(10, 7))
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1], hue=colData['condition'])
plt.title("PCA of RNA-Seq Data")
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.2f}%)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.2f}%)")
plt.show()

In [ ]:
import numpy as np

# Volcano plot of DEGs
plt.figure(figsize=(10, 7))
plt.scatter(res['log2FoldChange'], -np.log10(res['padj']), c='gray')
plt.scatter(sig_res['log2FoldChange'], -np.log10(sig_res['padj']), c='red')
plt.title("Volcano Plot")
plt.xlabel("Log2 Fold Change")
plt.ylabel("-log10 Adjusted P-value")
plt.show()

In [ ]:
import seaborn as sns

# Plot heatmap of significant DEGs
sns.heatmap(sig_res.head(50), cmap="RdBu_r")